In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev

In [ ]:
! pip install -q pytorch-lightning==1.1.8

In [ ]:
package_paths = ['../input/pytorch-library/pytorch_library/pytorch-image-models-master',]
import sys;
for pth in package_paths:
    sys.path.append(pth)
    
!pip install -q timm
import timm
import os
import numpy as np
import torch
import pandas as pd
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
from torchvision import transforms as tsfm
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.metrics import Metric

import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tqdm import tqdm
import torch
import torchvision.models as models
import albumentations as A
import torch.nn as nn
import random
from sklearn.model_selection import KFold, StratifiedKFold
from torchmetrics import AUC

In [ ]:
class ClassificationDataset:
    
    def __init__(self, image_paths, targets): 
        self.image_paths = image_paths
        self.targets = targets

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item])
        image = np.vstack(image).transpose((1, 0)).astype(np.float32)[np.newaxis, :, :]
        targets = self.targets[item]
                
        return torch.tensor(image, dtype=torch.float), torch.tensor(targets, dtype=torch.long)

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')

In [ ]:
X = df.img_path.values
Y = df.target.values
skf = StratifiedKFold(n_splits=5)
fold = 0

for train_index, test_index in skf.split(X, Y):
    train_images, valid_images = X[train_index], X[test_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]
    break

In [ ]:
epochs = 12
Batch_Size = 4

train_dataset = ClassificationDataset(image_paths=train_images, targets=train_targets)
valid_dataset = ClassificationDataset(image_paths=valid_images, targets=valid_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size,shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size,shuffle=False, num_workers=4)

In [ ]:
import torch
from torch.nn import functional as F
from torchvision import transforms
import pytorch_lightning as pl

class ImageClassifier(pl.LightningModule):

    def __init__(self):
        super(ImageClassifier, self).__init__()
        # not the best model...
        self.model = timm.create_model("resnet34", pretrained=True, num_classes=1, in_chans=1)
        self.criterion = F.cross_entropy 
        #self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        # called with self(x)
        return self.model(x)
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        logs = {'train_loss': loss}
        self.log_dict(logs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        logs = {'val_loss': loss}
        self.log_dict(logs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss


    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        logs = {'test_loss': loss}
        self.log_dict(logs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss


    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
from pytorch_lightning.loggers import CSVLogger
class CFG:
    seed = 77
    num_classes = 5
    num_epochs = 17
    batch_size = 64
    t_max = 16
    lr = 1e-3
    min_lr = 1e-6
    n_fold = 6
    num_workers = 8
    accum_grad_batch = 1
    early_stop_delta = 1e-7
    
logger = CSVLogger(save_dir=f'logs/', name="resnet34")
logger.log_hyperparams(CFG.__dict__)

In [ ]:
trainer = pl.Trainer(max_epochs=1, tpu_cores=8, logger=logger,
                      weights_summary='top',)

"""
Init model
"""
model = ImageClassifier()
"""
Fit(train) the model
"""
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=valid_loader)